# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.readthedocs.io/) and [Doxygen documentation](https://codedocs.xyz/CEED/libCEED/).

## Setting up libCEED for Python

Install libCEED for Python by running

In [1]:
! python -m pip install libceed

## CeedBasis

Here we show some basic examples to illustrate the `CeedBasis` class. In libCEED, a `CeedBasis` defines the finite element basis and associated quadrature rule (see [the API documentation](https://libceed.readthedocs.io/en/latest/libCEEDapi.html#finite-element-operator-decomposition)).

First we declare some auxiliary functions needed in the following examples

In [2]:
def eval(dim, x):
  result, center = 1, 0.1
  for d in range(dim):
    result *= math.tanh(x[d] - center)
    center += 0.1
  return result

def feval(x1, x2):
  return x1*x1 + x2*x2 + x1*x2 + 1

def dfeval(x1, x2):
  return 2*x1 + x2

* In the following example, we illustrate the creation and destruction of a H1Lagrange basis with two different quadrature rules (Gauss-Lobatto and Gauss, respectively) 

In [3]:
import libceed
import numpy as np

ceed = libceed.Ceed()

b = ceed.BasisTensorH1Lagrange(1, 1, 4, 4, libceed.GAUSS_LOBATTO)
print(b)
del b

b = ceed.BasisTensorH1Lagrange(1, 1, 4, 4, libceed.GAUSS)
print(b)
del b

CeedBasis: dim=1 P=4 Q=4
      qref1d:	 -1.00000000	 -0.44721360	  0.44721360	  1.00000000
   qweight1d:	  0.16666667	  0.83333333	  0.83333333	  0.16666667
    interp1d[0]:	  1.00000000	  0.00000000	  0.00000000	  0.00000000
    interp1d[1]:	  0.00000000	  1.00000000	  0.00000000	  0.00000000
    interp1d[2]:	  0.00000000	  0.00000000	  1.00000000	  0.00000000
    interp1d[3]:	  0.00000000	  0.00000000	  0.00000000	  1.00000000
      grad1d[0]:	 -3.00000000	  4.04508497	 -1.54508497	  0.50000000
      grad1d[1]:	 -0.80901699	  0.00000000	  1.11803399	 -0.30901699
      grad1d[2]:	  0.30901699	 -1.11803399	  0.00000000	  0.80901699
      grad1d[3]:	 -0.50000000	  1.54508497	 -4.04508497	  3.00000000

CeedBasis: dim=1 P=4 Q=4
      qref1d:	 -0.86113631	 -0.33998104	  0.33998104	  0.86113631
   qweight1d:	  0.34785485	  0.65214515	  0.65214515	  0.34785485
    interp1d[0]:	  0.62994317	  0.47255875	 -0.14950343	  0.04700152
    interp1d[1]:	 -0.07069480	  0.97297619	  0.13253993	 -0.0348

* In the following example, we illustrate how to get the number of nodes and number of quadrature points for a basis

In [4]:
b = ceed.BasisTensorH1Lagrange( 3, 1, 4, 5, libceed.GAUSS_LOBATTO)

p = libceed.Basis.get_num_nodes(b)
print('p =', p)

q = libceed.Basis.get_num_quadrature_points(b)
print('q =', q)

p = 64
q = 125


* In the following example, we demonstrate the application of an interpolatory basis in multiple dimensions

In [5]:
import math

for dim in range(1, 4):
  Q = 4
  Qdim = Q**dim
  Xdim = 2**dim
  x = np.empty(Xdim*dim, dtype="float64")
  uq = np.empty(Qdim, dtype="float64")

  for d in range(dim):
    for i in range(Xdim):
      x[d*Xdim + i] = 1 if (i % (2**(dim-d))) // (2**(dim-d-1)) else -1

  X = ceed.Vector(Xdim*dim)
  X.set_array(x, cmode=libceed.USE_POINTER)
  Xq = ceed.Vector(Qdim*dim)
  Xq.set_value(0)
  U = ceed.Vector(Qdim)
  U.set_value(0)
  Uq = ceed.Vector(Qdim)

  bxl = ceed.BasisTensorH1Lagrange(dim, dim, 2, Q, libceed.GAUSS_LOBATTO)
  bul = ceed.BasisTensorH1Lagrange(dim, 1, Q, Q, libceed.GAUSS_LOBATTO)

  bxl.apply(1, libceed.EVAL_INTERP, X, Xq)

  with Xq.array_read() as xq:
    for i in range(Qdim):
      xx = np.empty(dim, dtype="float64")
      for d in range(dim):
        xx[d] = xq[d*Qdim + i]
      uq[i] = eval(dim, xx)

  Uq.set_array(uq, cmode=libceed.USE_POINTER)

  # This operation is the identity because the quadrature is collocated
  bul.T.apply(1, libceed.EVAL_INTERP, Uq, U)

  bxg = ceed.BasisTensorH1Lagrange(dim, dim, 2, Q, libceed.GAUSS)
  bug = ceed.BasisTensorH1Lagrange(dim, 1, Q, Q, libceed.GAUSS)

  bxg.apply(1, libceed.EVAL_INTERP, X, Xq)
  bug.apply(1, libceed.EVAL_INTERP, U, Uq)

  with Xq.array_read() as xq, Uq.array_read() as u:
    print('xq =', xq)
    print('u =', u)


xq = [-0.86113631 -0.33998104  0.33998104  0.86113631]
u = [-0.75605827 -0.40905546  0.23605296  0.64590748]
xq = [-0.86113631 -0.86113631 -0.86113631 -0.86113631 -0.33998104 -0.33998104
 -0.33998104 -0.33998104  0.33998104  0.33998104  0.33998104  0.33998104
  0.86113631  0.86113631  0.86113631  0.86113631 -0.86113631 -0.33998104
  0.33998104  0.86113631 -0.86113631 -0.33998104  0.33998104  0.86113631
 -0.86113631 -0.33998104  0.33998104  0.86113631 -0.86113631 -0.33998104
  0.33998104  0.86113631]
u = [ 0.60441447  0.3677972  -0.10759824 -0.43761257  0.32701056  0.19899188
 -0.05821462 -0.23676457 -0.18870745 -0.11483191  0.03359382  0.13662934
 -0.5163568  -0.31421251  0.09192215  0.37385641]
xq = [-0.86113631 -0.86113631 -0.86113631 -0.86113631 -0.86113631 -0.86113631
 -0.86113631 -0.86113631 -0.86113631 -0.86113631 -0.86113631 -0.86113631
 -0.86113631 -0.86113631 -0.86113631 -0.86113631 -0.33998104 -0.33998104
 -0.33998104 -0.33998104 -0.33998104 -0.33998104 -0.33998104 -0.3399810

* In the following example, we demonstrate the application of the gradient of the shape functions in multiple dimensions

In [6]:
for dim in range (1, 4):
  P, Q = 8, 10
  Pdim = P**dim
  Qdim = Q**dim
  Xdim = 2**dim
  sum1 = sum2 = 0
  x = np.empty(Xdim*dim, dtype="float64")
  u = np.empty(Pdim, dtype="float64")

  for d in range(dim):
    for i in range(Xdim):
      x[d*Xdim + i] = 1 if (i % (2**(dim-d))) // (2**(dim-d-1)) else -1

  X = ceed.Vector(Xdim*dim)
  X.set_array(x, cmode=libceed.USE_POINTER)
  Xq = ceed.Vector(Pdim*dim)
  Xq.set_value(0)
  U = ceed.Vector(Pdim)
  Uq = ceed.Vector(Qdim*dim)
  Uq.set_value(0)
  Ones = ceed.Vector(Qdim*dim)
  Ones.set_value(1)
  Gtposeones = ceed.Vector(Pdim)
  Gtposeones.set_value(0)

  # Get function values at quadrature points
  bxl = ceed.BasisTensorH1Lagrange(dim, dim, 2, P, libceed.GAUSS_LOBATTO)
  bxl.apply(1, libceed.EVAL_INTERP, X, Xq)

  with Xq.array_read() as xq:
    for i in range(Pdim):
      xx = np.empty(dim, dtype="float64")
      for d in range(dim):
        xx[d] = xq[d*Pdim + i]
      u[i] = eval(dim, xx)

  U.set_array(u, cmode=libceed.USE_POINTER)

  # Calculate G u at quadrature points, G' * 1 at dofs
  bug = ceed.BasisTensorH1Lagrange(dim, 1, P, Q, libceed.GAUSS)
  bug.apply(1, libceed.EVAL_GRAD, U, Uq)
  bug.T.apply(1, libceed.EVAL_GRAD, Ones, Gtposeones)

  # Check if 1' * G * u = u' * (G' * 1)
  with Gtposeones.array_read() as gtposeones, Uq.array_read() as uq:
    for i in range(Pdim):
      sum1 += gtposeones[i]*u[i]
    for i in range(dim*Qdim):
      sum2 += uq[i]

  # Check that (1' * G * u - u' * (G' * 1)) is numerically zero
  print('1T * G * u - uT * (GT * 1) =', math.fabs(sum1 - sum2))

1T * G * u - uT * (GT * 1) = 1.7763568394002505e-15
1T * G * u - uT * (GT * 1) = 2.3092638912203256e-14
1T * G * u - uT * (GT * 1) = 9.947598300641403e-14


### Advanced topics

* In the following example, we demonstrate the QR factorization of a basis matrix

In [7]:
qr = np.array([1, -1, 4, 1, 4, -2, 1, 4, 2, 1, -1, 0], dtype="float64")
tau = np.empty(3, dtype="float64")

qr, tau = libceed.Basis.qr_factorization(ceed, qr, tau, 4, 3)

print('qr =')
for i in range(len(qr)):
  if qr[i] <= 1E-14  and qr[i] >= -1E-14:
    qr[i] = 0
  print('%12.8f'%qr[i])

print('tau =')
for i in range(len(tau)):
  if tau[i] <= 1E-14  and tau[i] >= -1E-14:
    tau[i] = 0
  print('%12.8f'%tau[i])

qr =
 -2.00000000
 -3.00000000
 -2.00000000
  0.33333333
 -5.00000000
  2.00000000
  0.33333333
  0.40000000
 -4.00000000
  0.33333333
 -0.20000000
 -0.50000000
tau =
  1.50000000
  1.66666667
  1.60000000


* In the following example, we demonstrate the symmetric Schur decomposition of a basis matrix

In [8]:
A = np.array([0.19996678, 0.0745459, -0.07448852, 0.0332866,
              0.0745459, 1., 0.16666509, -0.07448852,
              -0.07448852, 0.16666509, 1., 0.0745459,
              0.0332866, -0.07448852, 0.0745459, 0.19996678], dtype="float64")

lam = libceed.Basis.symmetric_schur_decomposition(ceed, A, 4)

print("Q =")
for i in range(4):
  for j in range(4):
    if A[j+4*i] <= 1E-14 and A[j+4*i] >= -1E-14:
       A[j+4*i] = 0
    print("%12.8f"%A[j+4*i])

print("lambda =")
for i in range(4):
  if lam[i] <= 1E-14 and lam[i] >= -1E-14:
    lam[i] = 0
  print("%12.8f"%lam[i])

Q =
  0.69153918
 -0.70710678
  0.14755868
  0.00004347
 -0.14721060
  0.00047835
  0.69240821
 -0.70632831
  0.14790715
  0.00047882
 -0.69066919
 -0.70788369
 -0.69153892
 -0.70710646
 -0.14755804
 -0.00100064
lambda =
  0.13487964
  0.23325338
  0.86513545
  1.16666509


* In the following example, we demonstrate the simultaneous diagonalization of a basis matrix

In [9]:
M = np.array([0.19996678, 0.0745459, -0.07448852, 0.0332866,
              0.0745459, 1., 0.16666509, -0.07448852,
              -0.07448852, 0.16666509, 1., 0.0745459,
              0.0332866, -0.07448852, 0.0745459, 0.19996678], dtype="float64")
K = np.array([3.03344425, -3.41501767, 0.49824435, -0.11667092,
              -3.41501767, 5.83354662, -2.9167733, 0.49824435,
              0.49824435, -2.9167733, 5.83354662, -3.41501767,
              -0.11667092, 0.49824435, -3.41501767, 3.03344425], dtype="float64")

x, lam = libceed.Basis.simultaneous_diagonalization(ceed, K, M, 4)

print("x =")
for i in range(4):
  for j in range(4):
    if x[j+4*i] <= 1E-14 and x[j+4*i] >= -1E-14:
      x[j+4*i] = 0
    print("%12.8f"%x[j+4*i])

print("lambda =")
for i in range(4):
  if lam[i] <= 1E-14 and lam[i] >= -1E-14:
    lam[i] = 0
  print("%12.8f"%lam[i])

x =
  1.68928908
  0.84673819
  0.51761264
  1.36958123
 -0.64827487
  0.54168848
  0.61211881
 -0.23257073
  0.64977850
 -0.54084624
  0.61207450
 -0.23180307
 -1.68991495
 -0.84558672
  0.51761799
  1.36951706
lambda =
 42.52259582
  2.46897852
  0.00000001
 15.01838167
